# Imports

In [3]:
import sys
sys.path.append('/groups/umcg-wijmenga/tmp01/projects/lude_vici_2021/rawdata/non-coding-somatic-mutations-in-cancer/Anne/scripts/')
from Database import Database
# import multiprocessing as mp
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats.distributions import chi2
from bioinfokit import analys, visuz
from scipy.stats import fisher_exact
import time
from scipy.special import factorial
import scipy.stats as stats
from scipy.stats import mannwhitneyu



from fisher import pvalue_npy
from scipy.stats import chi2_contingency
from scipy.stats import uniform, randint

# Get Data

In [ ]:
# path_db = 'D:/Hanze_Groningen/STAGE/lastdb/db_laatste_copy.db' #'D:/Hanze_Groningen/STAGE/DATAB/copydatabase_C.db'
# # Database connection
# db = Database(path_db)
# df_whole = pd.read_sql('''SELECT project.cancer, sum_dosage_GT.donor_project_ID, 
#                             sum_dosage_GT.donor_ID, sum_dosage_GT.snp_ID, sum_dosage_GT.GT2,
#                             snp.chr, snp.pos_start, snp.pos_end, snp.ref, snp.alt,
#                             snp.UCNE, snp.TFBS, snp.DNase, 
#                             snp.in_transcript, snp.in_coding, snp.in_exon,
#                             snp.before_gene, snp.after_gene, 
#                             snp.ID_eQTL, snp.eQTL, snp.close_eQTL_3000
#                     FROM project, sum_dosage_GT, snp 
#                     WHERE sum_dosage_GT.snp_ID=snp.ID AND 
#                               sum_dosage_GT.donor_project_ID = project.ID AND 
#                               (sum_dosage_GT.GT2 = 1 OR sum_dosage_GT.GT2 = 2) AND 
#                               sum_dosage_GT.total_read_count_sum >= 33;''', db.mydb_connection)
# db.close()
# df_whole.to_csv("D:/Hanze_Groningen/STAGE/lastdb/df_whole.tsv", sep='\t', encoding='utf-8', index=False)
# df_whole

In [ ]:
# df_whole['snp'] = df_whole['chr'].map(str) + '_' + df_whole['pos_start'].map(str) + '_' + df_whole['pos_end'].map(str)
# df_whole.to_csv("D:/Hanze_Groningen/STAGE/lastdb/df_whole_SNP.tsv", sep='\t', encoding='utf-8', index=False)
# df_whole

In [ ]:
df_whole = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/df_whole.tsv", sep='\t')
# df_whole = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/df_whole_SNP.tsv", sep='\t')
df_whole

# Filter data

In [ ]:
# Delete MT chromosoom
df_whole = df_whole.loc[df_whole['chr'] != 'MT']
print(set(df_whole['chr']))

In [ ]:
df_whole.dtypes

### non-coding, coding

In [ ]:
# #0=false
# noncoding_df = df_whole.loc[(df_whole['in_transcript'] == 0) & (df_whole['in_coding'] == 0) & (df_whole['in_exon'] == 0)]
# noncoding_df.to_csv("D:/Hanze_Groningen/STAGE/lastdb/noncoding_df.tsv", sep='\t', encoding='utf-8', index=False)
# noncoding_df

In [ ]:
# coding_df = df_whole.loc[(df_whole['in_transcript'] == 1) | (df_whole['in_coding'] == 1) | (df_whole['in_exon'] == 1)]
# coding_df.to_csv("D:/Hanze_Groningen/STAGE/lastdb/coding_df.tsv", sep='\t', encoding='utf-8', index=False)
# coding_df

In [ ]:
noncoding_df = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/noncoding_df.tsv", sep='\t')
coding_df = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/coding_df.tsv", sep='\t')

### breast, nonbreast

In [ ]:
# all_breast =  df_whole.loc[df_whole['cancer'] == 'Breast']
# all_breast.to_csv("D:/Hanze_Groningen/STAGE/lastdb/all_breast.tsv", sep='\t', encoding='utf-8', index=False)

# noncoding_breast = noncoding_df.loc[noncoding_df['cancer'] == 'Breast']
# noncoding_breast.to_csv("D:/Hanze_Groningen/STAGE/lastdb/noncoding_breast.tsv", sep='\t', encoding='utf-8', index=False)

# coding_breast = coding_df.loc[coding_df['cancer'] == 'Breast']
# coding_breast.to_csv("D:/Hanze_Groningen/STAGE/lastdb/coding_breast.tsv", sep='\t', encoding='utf-8', index=False)


In [ ]:
# all_nonbreast = df_whole.loc[df_whole['cancer'] != 'Breast']
# all_nonbreast.to_csv("D:/Hanze_Groningen/STAGE/lastdb/all_nonbreast.tsv", sep='\t', encoding='utf-8', index=False)

# noncoding_nonbreast = noncoding_df.loc[noncoding_df['cancer'] != 'Breast']
# noncoding_nonbreast.to_csv("D:/Hanze_Groningen/STAGE/lastdb/noncoding_nonbreast.tsv", sep='\t', encoding='utf-8', index=False)

# coding_nonbreast = coding_df.loc[coding_df['cancer'] != 'Breast']
# coding_nonbreast.to_csv("D:/Hanze_Groningen/STAGE/lastdb/coding_nonbreast.tsv", sep='\t', encoding='utf-8', index=False)

In [4]:
# all_breast = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/all_breast.tsv", sep='\t')
noncoding_breast = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/noncoding_breast.tsv", sep='\t')
# coding_breast = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/coding_breast.tsv", sep='\t')

# all_nonbreast = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/all_nonbreast.tsv", sep='\t')
noncoding_nonbreast = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/noncoding_nonbreast.tsv", sep='\t')
# coding_nonbreast = pd.read_csv("D:/Hanze_Groningen/STAGE/lastdb/coding_nonbreast.tsv", sep='\t')

C:\Users\Anne_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3397: DtypeWarning: Columns (5,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Testen

In [ ]:
def chi_square_self(df, colname_b, colname_nb, num_donor_b, num_donor_nb):
    df['counts_breast_plus'] = df[colname_b] + (1/num_donor_b)
    print(set(df['counts_breast_plus']))
    df['counts_nonbreast_plus'] = df[colname_nb] + (1/num_donor_nb)
    print(set(df['counts_nonbreast_plus']))
    n = np.array([num_donor_b, num_donor_nb])
    print(n)
    S = df[['counts_breast_plus', 'counts_nonbreast_plus']].to_numpy()
    f = n / n.sum()
    E1 = S.sum(axis=1)[:,None]*f
    C = (n-S)
    print(set(C[0]))
    print(set(C[1]))
    E2 = C.sum(axis=1)[:,None]*f
    D1 = ((E1-S)**2/E1).sum(axis=1)
    D2 = ((E2-C)/E2).sum(axis=1)
    X2 = D1 + D2
    #
    p_value_X2 = chi2.sf(X2,1)
    #
    log10_p_value = -np.log10(p_value_X2)
    log10_p_value
    #
    df['X2'] = X2
    df['p_value_X2_self'] = p_value_X2
    df['log10_p_value_X2_self'] = log10_p_value
    #
    S_C = np.concatenate((S, C), axis=1)
    return df, S_C, n, S, C    

In [ ]:
def chi_square(S_C, df):
    start_time = time.perf_counter()

    p_value_X2_TEST = list()
    for index, value in enumerate(S_C):
        table = np.array([[value[0], value[2]], [value[1], value[3]]])
        stat, p, dof, expected = chi2_contingency(table)
        p_value_X2_TEST.append(p)
        if (index%1000000) == 0:
            print(len(p_value_X2_TEST))
    print(len(p_value_X2_TEST))

    print("--- %s seconds ---" % (time.perf_counter() - start_time))
    df['p_value_X2'] = p_value_X2_TEST
    return df
    

In [ ]:
def log2_fc(df, n, S):
    constant = np.log(n[0]) - np.log(n[1])
    log2_fc = (np.log(S[:, 0]) - np.log(S[:, 1]) - constant) / np.log(2)
    df['log2_fc'] = log2_fc # log2(FC)
    return df

In [ ]:
def fisher_test(S_C, df):
    start_time = time.perf_counter()

    p_value_F = list()
    for index, value in enumerate(S_C):
        table = np.array([[value[0], value[2]], [value[1], value[3]]])
        oddsr, p = fisher_exact(table, alternative='two-sided')
        p_value_F.append(p)
        if (index%1000000) == 0:
            print(len(p_value_F))
    print(len(p_value_F))

    print("--- %s seconds ---" % (time.perf_counter() - start_time))
    df['p_value_F'] = p_value_F
    return df

In [ ]:
def shapiro_test(df, column_name, column_name_filter):
    """
    As the p value obtained from the Shapiro-Wilk test is significant (p < 0.05), 
    we conclude that the data is not normally distributed. Further, in histogram data 
    distribution shape does not look normal. Therefore, Mann-Whitney U test is more appropriate for analyzing two samples.
    """
    w, pvalue = stats.shapiro(df[column_name])
    print(f'w:{w}, pvalue:{pvalue}')
    # plot histogram
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle('Frequency histogram')
    ax1.hist(df[column_name_filter], bins=df[column_name].max(), histtype='bar', ec='k') 
    ax2.hist(df[column_name], bins=df[column_name].max(), histtype='bar', ec='k') 
    plt.show()

In [ ]:
def mannwhitney(df, column_breast, column_nonbreast):
    """
    Mann-Whitney U test interpretation: As the p value obtained from the Mann-Whitney U test 
    is significant (U = 489.5, p < 0.05), we conclude that the yield of the two genotypes 
    significantly different from each other .
    """
    # U1, p = mannwhitneyu(list(breast['filter_snps']), list(nonbreast['filter_snps']), alternative = 'two-sided')
    print(mannwhitneyu(list(df[column_breast]), list(df[column_nonbreast]), alternative = 'two-sided'))


In [ ]:
def volcano_plot(df, p_value_column):
    plt = visuz.GeneExpression.volcano(df=df,lfc='log2_fc',pv=p_value_column,show=True)
    

In [ ]:
def all_test(df, num_donor_b, num_donor_nb, type_df, type_analyse):
    print('\nboxplot')
    df.boxplot(column=['counts_breast'], grid=False)
    df.boxplot(column=['counts_nonbreast'], grid=False)
    
    print('\nfilter columns (divide by max)')
    df['filter_snps_b'] = df['counts_breast']/ num_donor_b
    df['filter_snps_nb'] = df['counts_nonbreast']/ num_donor_nb
    
    print('\nshapiro_test')
    shapiro_test(df, 'counts_breast', 'filter_snps_b')
    shapiro_test(df, 'counts_nonbreast', 'filter_snps_nb')
    
    print('\nmannwhitney')
    mannwhitney(df, 'filter_snps_b', 'filter_snps_nb')
    
#     print('\ntests')
#     df, S_C, n, S, C = chi_square_self(df, 'counts_breast', 'counts_nonbreast', num_donor_b, num_donor_nb)
#     df = chi_square(S_C, df)
#     df = log2_fc(df, n, S)
#     df = fisher_test(S_C, df)
#     df = log2_fc(df, n, S)
#     df.to_csv(f"D:/Hanze_Groningen/STAGE/lastdb/{type_analyse}_{type_df}_both_0_TESTS.tsv", sep='\t', encoding='utf-8', index=False)
    
#     print('\nvolcano_plot')
#     volcano_plot(sort_snp_count_all_both_0, 'p_value_X2_self')
#     volcano_plot(sort_snp_count_all_both_0, 'p_value_X2')
#     volcano_plot(sort_snp_count_all_both_0, 'p_value_F')
    
    return df

# per SNP

### TEST

In [ ]:
def make_snp_df(df, type_df, type_c):
    print('----------')
    df['snp'] = df['chr'].map(str) + '_' + df['pos_start'].map(str) + '_' + df['pos_end'].map(str)
    print(len(list(set(df['donor_ID']))))
    num_donor = len(list(set(df['donor_ID'])))

    df_dict = dict(Counter(list(df['snp'])))
    
    snp_count_df = pd.DataFrame([df_dict.keys(), df_dict.values()]).T
    snp_count_df.columns= ['snp', 'counts']
    max_df = snp_count_df['counts'].max()
    print(snp_count_df['counts'].max())

    snp_count_df[['chr', 'pos_start', 'pos_end']] = snp_count_df['snp'].str.split('_', expand=True)

    snp_count_df.drop('snp', axis=1, inplace=True)

    sort_snp_count_df = snp_count_df.sort_values('counts', ascending=False)

    sort_snp_count_df.rename(columns={'counts': f'counts_{type_c}'}, inplace=True)
    
    df_R = sort_snp_count_df.reset_index().drop([f'counts_{type_c}', 'index'], 1)
    df_R['chr'] ='chr' + df_R['chr'].astype(str)
    df_R.to_csv(f"D:/Hanze_Groningen/STAGE/lastdb/R/{type_df}_{type_c}.tsv", sep='\t', encoding='utf-8', header=None)
    return num_donor, sort_snp_count_df

In [ ]:
# df_whole -- # noncoding_df # coding_df -- # all_breast # noncoding_breast # coding_breast --
# all_nonbreast # noncoding_nonbreast # coding_nonbreast

In [ ]:
def run_snp_tests(df_breast, df_nonbreast, type_df, type_analyse):
    print('\nset snps')
    num_donor_b, sort_snp_count_breast = make_snp_df(df_breast, type_df, 'breast')
    num_donor_nb, sort_snp_count_nonbreast = make_snp_df(all_nonbreast, type_df, 'nonbreast')
    
    print('\nmerge dfs')
    sort_snp_count_both = sort_snp_count_breast.merge(sort_snp_count_nonbreast, on=['chr', 'pos_start', 'pos_end'], how='outer')
    sort_snp_count_both_0 = sort_snp_count_both.fillna(0)
    sort_snp_count_both_0.to_csv(f"D:/Hanze_Groningen/STAGE/lastdb/{type_analyse}_{type_df}_both_0.tsv", sep='\t', encoding='utf-8', index=False)
    
    sort_snp_count_both_0 = all_test(sort_snp_count_both_0, num_donor_b, num_donor_nb, type_df, type_analyse)
    
#     print('\nboxplot')
#     sort_snp_count_both_0.boxplot(column=['counts_breast'], grid=False)
#     sort_snp_count_both_0.boxplot(column=['counts_nonbreast'], grid=False)
    
#     print('\nfilter columns (divide by max)')
#     sort_snp_count_both_0['filter_snps_b'] = sort_snp_count_both_0['counts_breast']/ num_donor_b
#     sort_snp_count_both_0['filter_snps_nb'] = sort_snp_count_both_0['counts_nonbreast']/ num_donor_nb
    
#     print('\nshapiro_test')
#     shapiro_test(sort_snp_count_both_0, 'counts_breast', 'filter_snps_b')
#     shapiro_test(sort_snp_count_both_0, 'counts_nonbreast', 'filter_snps_nb')
    
#     print('\nmannwhitney')
#     mannwhitney(sort_snp_count_both_0, 'filter_snps_b', 'filter_snps_nb')
    
#     print('\ntests')
#     sort_snp_count_both_0, S_C, n, S, C = chi_square_self(sort_snp_count_both_0, 'counts_breast', 'counts_nonbreast', num_donor_b, num_donor_nb)
#     sort_snp_count_both_0 = chi_square(S_C, sort_snp_count_both_0)
#     sort_snp_count_both_0 = log2_fc(sort_snp_count_both_0, n, S)
#     sort_snp_count_both_0 = fisher_test(S_C, sort_snp_count_both_0)
#     sort_snp_count_both_0 = log2_fc(sort_snp_count_both_0, n, S)
#     sort_snp_count_both_0.to_csv(f"D:/Hanze_Groningen/STAGE/lastdb/sort_snp_count_{type_df}_both_0_TESTS.tsv", sep='\t', encoding='utf-8', index=False)
    
#     print('\nvolcano_plot')
#     volcano_plot(sort_snp_count_all_both_0, 'p_value_X2_self')
#     volcano_plot(sort_snp_count_all_both_0, 'p_value_X2')
#     volcano_plot(sort_snp_count_all_both_0, 'p_value_F')
    
#     return sort_snp_count_both_0, num_donor_b, num_donor_nb 

### ALL (all_breast, all_nonbreast)

In [ ]:
# sort_snp_count_both_0 = run_snp_tests(all_breast, all_nonbreast, 'ALL')

In [ ]:
num_donor_b, sort_snp_count_all_breast = make_snp_df(all_breast, 'All', 'breast')
num_donor_nb, sort_snp_count_all_nonbreast = make_snp_df(all_nonbreast, 'All', 'nonbreast')

In [ ]:
sort_snp_count_all_both = sort_snp_count_all_breast.merge(sort_snp_count_all_nonbreast, on=['chr', 'pos_start', 'pos_end'], how='outer')
sort_snp_count_all_both_0 = sort_snp_count_all_both.fillna(0)
sort_snp_count_all_both_0.to_csv(f"D:/Hanze_Groningen/STAGE/lastdb/sort_snp_count_all_both_0.tsv", sep='\t', encoding='utf-8', index=False)
sort_snp_count_all_both_0

In [ ]:
sort_snp_count_all_both_0.boxplot(column=['counts_breast'], grid=False)

In [ ]:
sort_snp_count_all_both_0.boxplot(column=['counts_nonbreast'], grid=False)

In [ ]:
sort_snp_count_all_both_0['filter_snps_b'] = sort_snp_count_all_both_0['counts_breast']/ num_donor_b
sort_snp_count_all_both_0['filter_snps_nb'] = sort_snp_count_all_both_0['counts_nonbreast']/ num_donor_nb

In [ ]:
shapiro_test(sort_snp_count_all_both_0, 'counts_breast', 'filter_snps_b')

In [ ]:
shapiro_test(sort_snp_count_all_both_0, 'counts_nonbreast', 'filter_snps_nb')

In [ ]:
mannwhitney(sort_snp_count_all_both_0, 'filter_snps_b', 'filter_snps_nb')

In [ ]:
set(sort_snp_count_all_both_0['counts_nonbreast'])

In [ ]:
sort_snp_count_all_both_0, S_C, n, S, C = chi_square_self(sort_snp_count_all_both_0, 'counts_breast', 'counts_nonbreast', num_donor_b, num_donor_nb)
# sort_snp_count_all_both_0 = chi_square(S_C, sort_snp_count_all_both_0)
# sort_snp_count_all_both_0 = log2_fc(sort_snp_count_all_both_0, n, S)
# sort_snp_count_all_both_0 = fisher_test(S_C, sort_snp_count_all_both_0)
# sort_snp_count_all_both_0 = log2_fc(sort_snp_count_all_both_0, n, S)
# sort_snp_count_all_both_0.to_csv(f"D:/Hanze_Groningen/STAGE/lastdb/R/sort_snp_count_all_both_0_TESTS.tsv", sep='\t', encoding='utf-8', index=False)

In [ ]:
set(S_C[0])

In [ ]:
sort_snp_count_all_both_0 = chi_square(S_C, sort_snp_count_all_both_0)

In [ ]:
sort_snp_count_all_both_0 = fisher_test(S_C, sort_snp_count_all_both_0)

In [ ]:
sort_snp_count_all_both_0 = log2_fc(sort_snp_count_all_both_0, n, S)

In [ ]:
sort_snp_count_all_both_0.to_csv(f"D:/Hanze_Groningen/STAGE/lastdb/sort_snp_count_all_both_0_TESTS.tsv", sep='\t', encoding='utf-8', index=False)

In [ ]:
volcano_plot(sort_snp_count_all_both_0, 'p_value_X2_self')

In [ ]:
volcano_plot(sort_snp_count_all_both_0, 'p_value_X2')

In [ ]:
volcano_plot(sort_snp_count_all_both_0, 'p_value_F')

### non-coding (noncoding_breast, noncoding_nonbreast)

In [ ]:
sort_snp_count_noncoding_both_0, num_donor_b, num_donor_nb  = run_snp_tests(noncoding_breast, noncoding_nonbreast, 'NonCoding')

### coding (coding_breast, coding_nonbreast)

In [ ]:
sort_snp_count_coding_both_0 = run_snp_tests(coding_breast, coding_nonbreast, 'Coding')

# position 2000

In [ ]:
# df_whole -- # noncoding_df # coding_df -- # all_breast # noncoding_breast # coding_breast --
# all_nonbreast # noncoding_nonbreast # coding_nonbreast

In [ ]:
def make_merge_df(path_breast, path_nonbreast):
    colnames=['index', 'counts_breast', 'chr', 'start_region', 'stop_region']
    breast = pd.read_csv(path_breast, sep='\t', header=None, names=colnames)
    breast.sort_values(['chr', 'start_region'], inplace=True)
    
    colnames=['index', 'counts_nonbreast', 'chr', 'start_region', 'stop_region']
    nonbreast = pd.read_csv(path_nonbreast, sep='\t', header=None, names=colnames)   
    nonbreast.sort_values(['chr', 'start_region'], inplace=True)
    
    merged_df = breast.merge(nonbreast, on=['chr', 'start_region', 'stop_region'], how='outer')
    merged_df.drop(columns=['index_x', 'index_y'], inplace=True)
    return merged_df

In [ ]:
def make_df_2000(merge_df):
    df = pd.DataFrame(columns=['counts_breast', 'chr', 'start_region', 'stop_region', 'counts_nonbreast', 'snps_b_double', 'snps_nb_double'])

    # chrom = ['chr4']
    for i in list(set(merge_df['chr'])):
        select_df = merge_df[merge_df['chr'] == i]
        select_df['snps_b_double'] = select_df['counts_breast'] + select_df['counts_breast'].shift(1)
        select_df['snps_nb_double'] = select_df['counts_nonbreast'] + select_df['counts_nonbreast'].shift(1)
        select_df['start_new'] = select_df['start_region'].shift(1) #.astype(str) #+ "_" + select_df['stop_region'].astype(str)
    #     select_df['stop_new'] = select_df['stop_region']       
        select_df = select_df.reset_index()
        select_df.drop(columns=['index'], inplace=True)
        select_df = select_df.reset_index()
        select_df = select_df[(select_df['index'] % 2 != 0) | (select_df['index'] == select_df['index'].max())]
        select_df['start_new'] = list(select_df['start_new'][:-1])+[list(select_df['start_region'])[-1]]
        select_df.drop(columns=['index'], inplace=True)
    #     select_df['stop_new'] = list(select_df['stop_new'][:-1])+[list(select_df['stop_region'])[-1]]
        df = pd.concat([df, select_df])
    df['start_region'] = df['start_new']
    df.drop(columns=['counts_breast', 'counts_nonbreast', 'start_new'], inplace=True)
    df.rename(columns = {'snps_b_double':'counts_breast', 'snps_nb_double':'counts_nonbreast'}, inplace = True)
    return df


In [ ]:
merge_df = make_merge_df("D:/Hanze_Groningen/STAGE/R/PLOTS/kary/vs/before/1000_filter/breast_ALL_1000.tsv", "D:/Hanze_Groningen/STAGE/R/PLOTS/kary/vs/before/1000_filter/nonbreast_ALL_1000.tsv")
# df_1000_tests = all_test(merge_df, num_donor_b, num_donor_nb, 'ALL', 'Region_1000')

In [ ]:
df_2000 = make_df_2000(merge_df)
df_2000_tests = all_test(df_2000, num_donor_b, num_donor_nb, 'ALL', 'Region_2000')

### ALL (all_breast, all_nonbreast)

### non-coding (noncoding_breast, noncoding_nonbreast)

### coding (coding_breast, coding_nonbreast)

# UCNE

In [ ]:
# df_whole -- # noncoding_df # coding_df -- # all_breast # noncoding_breast # coding_breast --
# all_nonbreast # noncoding_nonbreast # coding_nonbreast

### ALL (all_breast, all_nonbreast)

### non-coding (noncoding_breast, noncoding_nonbreast)

### coding (coding_breast, coding_nonbreast)

# TFBS

In [ ]:
# df_whole -- # noncoding_df # coding_df -- # all_breast # noncoding_breast # coding_breast --
# all_nonbreast # noncoding_nonbreast # coding_nonbreast

### ALL (all_breast, all_nonbreast)

### non-coding (noncoding_breast, noncoding_nonbreast)

### coding (coding_breast, coding_nonbreast)

# DNase

In [ ]:
# df_whole -- # noncoding_df # coding_df -- # all_breast # noncoding_breast # coding_breast --
# all_nonbreast # noncoding_nonbreast # coding_nonbreast

### ALL (all_breast, all_nonbreast)

### non-coding (noncoding_breast, noncoding_nonbreast)

### coding (coding_breast, coding_nonbreast)

# before_gene (promotor)

In [ ]:
# df_whole -- # noncoding_df # coding_df -- # all_breast # noncoding_breast # coding_breast --
# all_nonbreast # noncoding_nonbreast # coding_nonbreast

### ALL (all_breast, all_nonbreast)

### non-coding (noncoding_breast, noncoding_nonbreast)

### coding (coding_breast, coding_nonbreast)

# eQTL

In [ ]:
# df_whole -- # noncoding_df # coding_df -- # all_breast # noncoding_breast # coding_breast --
# all_nonbreast # noncoding_nonbreast # coding_nonbreast

### ALL (all_breast, all_nonbreast)

### non-coding (noncoding_breast, noncoding_nonbreast)

### coding (coding_breast, coding_nonbreast)

# Genotype

In [ ]:
# df_whole -- # noncoding_df # coding_df -- # all_breast # noncoding_breast # coding_breast --
# all_nonbreast # noncoding_nonbreast # coding_nonbreast

In [23]:
import statsmodels.api as sm

### ALL (all_breast, all_nonbreast)

In [ ]:
# all_breast, all_nonbreast

In [5]:
print(noncoding_breast.columns)
print(set(noncoding_breast['GT2']))
noncoding_breast

Index(['cancer', 'donor_project_ID', 'donor_ID', 'snp_ID', 'GT2', 'chr',
       'pos_start', 'pos_end', 'ref', 'alt', 'UCNE', 'TFBS', 'DNase',
       'in_transcript', 'in_coding', 'in_exon', 'before_gene', 'after_gene',
       'ID_eQTL', 'eQTL', 'close_eQTL_3000'],
      dtype='object')
{1, 2}


,cancer,donor_project_ID,donor_ID,snp_ID,GT2,chr,pos_start,pos_end,ref,alt,...,TFBS,DNase,in_transcript,in_coding,in_exon,before_gene,after_gene,ID_eQTL,eQTL,close_eQTL_3000
0,Breast,25,1305,1922329,1,7,50288406,50288406,A,-,...,0,0,0,0,0,0,0,NaN,0,0
1,Breast,25,1305,2129052,1,18,73873870,73873870,C,T,...,0,0,0,0,0,0,0,NaN,0,0
2,Breast,25,1305,3656796,1,1,9042655,9042655,C,T,...,1,0,0,0,0,0,0,NaN,0,0
3,Breast,25,1305,3656801,1,1,11669158,11669158,G,C,...,0,0,0,0,0,0,0,NaN,0,0
4,Breast,25,1305,3656804,1,1,11669165,11669165,G,A,...,0,0,0,0,0,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340078,Breast,28,2056,8740751,1,11,1162014,1162014,C,T,...,0,1,0,0,0,0,0,NaN,0,0
340079,Breast,28,2056,8740769,1,X,105937522,105937522,G,A,...,0,0,0,0,0,0,0,NaN,0,0
340080,Breast,28,2056,8740788,1,10,52751290,52751290,C,T,...,1,1,0,0,0,0,0,NaN,0,0
340081,Breast,28,2056,8740796,1,11,60511341,60511341,C,G,...,0,0,0,0,0,0,0,NaN,0,0


In [6]:
def make_GT_df(df, type_c, num_donors):
    select_df = df[['GT2', 'snp_ID', 'chr', 'pos_start', 'pos_end']]
    select_1_df = select_df[select_df['GT2'] == 1]
    select_2_df = select_df[select_df['GT2'] == 2]
    GT1_df = select_1_df.groupby(by=['snp_ID', 'chr', 'pos_start', 'pos_end']).sum().rename(columns = {'GT2':f'GT_1_{type_c}'})
    GT2_df = select_2_df.groupby(by=['snp_ID', 'chr', 'pos_start', 'pos_end']).sum().rename(columns = {'GT2':f'GT_2_{type_c}'})
    GT_df = GT1_df.merge(GT2_df, on=['snp_ID', 'chr', 'pos_start', 'pos_end'], how='outer')
    GT_df_0 = GT_df.fillna(0)
    GT_df_0[f'GT_2_{type_c}'] = GT_df_0[f'GT_2_{type_c}'] /2
    GT_df_0.sort_values(by=['chr', 'pos_start'], inplace = True)
    GT_df_0[f'GT_0_{type_c}'] = num_donors - (GT_df_0[f'GT_1_{type_c}']+GT_df_0[f'GT_2_{type_c}'])
    return GT_df_0

In [7]:
breast_GT = make_GT_df(noncoding_breast, 'b', 286)
nonbreast_GT = make_GT_df(noncoding_nonbreast, 'nb', 1952)
both_GT = breast_GT.merge(nonbreast_GT, on=['snp_ID', 'chr', 'pos_start', 'pos_end'], how='outer')
both_GT

,,,,GT_1_b,GT_2_b,GT_0_b,GT_1_nb,GT_2_nb,GT_0_nb
snp_ID,chr,pos_start,pos_end,,,,,,
8070609,1,565361,565361,1.0,0.0,285.0,NaN,NaN,NaN
8044916,1,568813,568813,1.0,0.0,285.0,NaN,NaN,NaN
8443261,1,729983,729983,1.0,0.0,285.0,NaN,NaN,NaN
8066911,1,748323,748323,1.0,0.0,285.0,NaN,NaN,NaN
4502730,1,797479,797479,1.0,0.0,285.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
13661977,Y,59306964,59306964,NaN,NaN,NaN,1.0,0.0,1951.0
13010824,Y,59311457,59311457,NaN,NaN,NaN,1.0,0.0,1951.0
13380994,Y,59327858,59327858,NaN,NaN,NaN,1.0,0.0,1951.0


In [19]:
# both_GT= both_GT.fillna(0).reset_index()
test = both_GT.head(3)
test

,level_0,index,snp_ID,chr,pos_start,pos_end,GT_1_b,GT_2_b,GT_0_b,GT_1_nb,GT_2_nb,GT_0_nb
0,0,0,8070609,1,565361,565361,1.0,0.0,285.0,0.0,0.0,0.0
1,1,1,8044916,1,568813,568813,1.0,0.0,285.0,0.0,0.0,0.0
2,2,2,8443261,1,729983,729983,1.0,0.0,285.0,0.0,0.0,0.0


In [27]:
for index, row in test.iterrows():
    # pd.crosstab([a,d], [b, c, ], rownames=['breast', 'nonbreast'], colnames=['0', '1', '2'])
    contingency_table = np.array(([row['GT_0_b'], row['GT_1_b'], row['GT_2_b']], [row['GT_0_nb'], row['GT_1_nb'], row['GT_2_nb']]))
    print(contingency_table)
    table = sm.stats.Table(contingency_table)
    print(table)
    p_value = table.test_ordinal_association().pvalue
    print(p_value)

[[285.   1.   0.]
 [  0.   0.   0.]]
A 2x3 contingency table with counts:
[[285.    1.    0.5]
 [  0.5   0.5   0.5]]
1.4102499348311139e-18
[[285.   1.   0.]
 [  0.   0.   0.]]
A 2x3 contingency table with counts:
[[285.    1.    0.5]
 [  0.5   0.5   0.5]]
1.4102499348311139e-18
[[285.   1.   0.]
 [  0.   0.   0.]]
A 2x3 contingency table with counts:
[[285.    1.    0.5]
 [  0.5   0.5   0.5]]
1.4102499348311139e-18


In [ ]:
# import statsmodels.api as sm
# table = sm.stats.Table(contingency_table)
# p_value = table.test_ordinal_association().pvalue
# contingency table was in dit geval een pandas crosstab
# maar kan sws ook met een numpy matrix
# https://www.statsmodels.org/stable/contingency_tables.html#independence

### non-coding (noncoding_breast, noncoding_nonbreast)

### coding (coding_breast, coding_nonbreast)